In [1]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split


Мною были распаршены несколько русскоязычных чатов, посвящённые яызкам программирования. При парсинге я учитывал только сниппеты, в которых больше английского, чем русского, потому что столкнулся с проблемой, что вставки с кодом использовались для цитат или условий задач (модель должна распознать такое, но поскольку я отмечаю это как код на языке чатика, это только ухудшает метрику на тесте). 

К сожалению, найти чат, по Mardown и Yaml не представилось возможным, поэтому в тестах на реальных сообщения они будут отсутствовать.

In [2]:
def count_russian_chars_fraction(text: str):
    return len(re.findall('[\u0400-\u04FF]', text)) / len(text)

In [3]:
CHATS_DUMP_PATH = os.path.join('..', 'data', 'tg_chats')
DATA_PATH = os.path.join('..', 'data', 'code_snippets')

for chat_name in os.listdir(CHATS_DUMP_PATH):
    if os.path.isdir(os.path.join(CHATS_DUMP_PATH, chat_name)):
        json = pd.read_json(os.path.join(CHATS_DUMP_PATH, chat_name, 'result.json'))
        messages = list(json['messages'])
        text_etities = list([m['text_entities'] for m in messages if m['type'] == 'message'])
        text_etities_flat = [item for sublist in text_etities for item in sublist]
        code_snippets = [d['text'] for d in text_etities_flat if d.get("type") == "pre" and count_russian_chars_fraction(d['text']) < 0.5]
        print(code_snippets[1:10])
        df = pd.DataFrame()
        df['snippets'] = code_snippets
        df.to_csv(os.path.join(DATA_PATH, chat_name + '.csv'))
        print(chat_name, "got", len(messages), 'messages, extracted', len(code_snippets), 'snippets')

['exec("sudo -u bratsk /usr/bin/sshpass -p admin /usr/bin/ssh -o StrictHostKeyChecking=no root@192.168.0.102 sed -i \'s/.*bitmain-freq1.*/\\"${bar1}\\"/\' /config/221 sed -i \'s/.*bitmain-freq3.*/\\"${bar3}\\"/\' /config/221");', '$bar1 = <<<EOT\n"\\""bitmain-freq1"\\"" : "\\""644466"\\"",\nEOT;\n\n$bar2 = <<<EOT\n"\\""bitmain-freq2"\\"" : "\\""666"\\"",\nEOT;\n\n$bar3 = <<<EOT\n"\\""bitmain-freq3"\\"" : "\\""666"\\"",\nEOT;', '"bitmain-freq1" : "625",\n"bitmain-freq2" : "625",\n"bitmain-freq3" : "618",', 'script_sql && \\\nscript_php && \\\nscript_jq & \nsleep 15 \nscript_sql2', 'script_sql && \\\nscript_php \nEXITCODE=$?\nif [[ $EXITCODE == 0 ]]; then\nscript_jq & \nscript_sql2 &\nfi', 'User "postgres" has no password assigned.\nRole "test" does not exist.', 'exec("sudo -u bratsk /usr/bin/sshpass -p admin /usr/bin/ssh -o StrictHostKeyChecking=no root@192.168.0.102 \'sed -i \'s/.*bitmain-freq1.*/\\"${bar1}\\"/\' /config/221\'", $output);', 'mount: /mnt: cannot remount /dev/sdb1 read-w

Также сделаем датасет из сообщений без кода из чатика, например, по Котлину.

In [4]:
json = pd.read_json(os.path.join(CHATS_DUMP_PATH, 'KOTLIN_1', 'result.json'))
messages = list(json['messages'])
text_etities = list([m['text_entities'] for m in messages if m['type'] == 'message'])
text_etities_flat = [item for sublist in text_etities for item in sublist]
text_snippets = [d['text'] for d in text_etities_flat if d.get("type") == "plain" and len(d['text']) > 40]
print(text_snippets[1:10])
df = pd.DataFrame()
df['snippets'] = text_snippets

train, test = train_test_split(df, test_size=0.1, train_size=0.4, random_state=0)

train['content'] = train['snippets']
train['type'] = 'other'
train = train.drop(columns=['snippets'])

train.to_csv(os.path.join('..', 'data','OTHER.csv'))
test.to_csv(os.path.join(DATA_PATH, 'OTHER_1.csv'))
print(chat_name, "got", len(messages), 'messages, extracted', len(text_snippets), 'snippets')
test, train

['Не обязательно. Ты же на сервере можешь куда посылать хттп запросы ещё', ' тут имеет смысл Http Server/Client. А вот Client уже может быть и на сервере, и на мобильном телефоне и на страничке на фронте', 'Я не думаю, что человека сильно интересует коммуникация между сервисами в рамках кластера, но в целом неправ, согласен.', 'Так то прочёл со своим плохеньким английским, что то про сторонние сервисы,  но зачем разделено на серверс и клиентс, не въехал.', 'Клиент -- штукая которая умеет делать хттп запросы, сервер -- отвечать на них.', 'Почему в рамках кластера то? Куча бекендов  делают запросы во вне, это очень распространенная практика', 'Ну там поступаться в АПИ Амазона/ВКонтакте/Гугла и ТД', 'Ладно, сдаюсь, я совсем уже к вечеру сварился и несу чушь.', 'Всем привет\n\n    companion object {\n        const val EXTRA_MESSAGE = "message"\n    }\n\nТут нужно писать const или так как и так static + val - уже все хорошо?']
PYTHON_1 got 119133 messages, extracted 72218 snippets


(                                                snippets
 37534  Var action = ""\nWhile action != "Exit"){\nAct...
 57223  Здраствуйте пишу простую программу хочу что ск...
 32913          Это уже из раздела про лямбды с ресивером
 27424  вероятнее всего что пройтись по всему массиву ...
 68081  Не надо спамить непроверенными ссылками в непр...
 ...                                                  ...
 5582   Модификатор inline не меняет семантику кода, е...
 69377  Не, я к тому, что интересно было сравнить прои...
 64924  Вроде бы уже были попытки создания таких униве...
 18585  Это может быть что угодно. нажмите на ссылочку...
 67004  Но проблема же в том, что у нас у андроида гла...
 
 [7222 rows x 1 columns],
                                                  content   type
 14405  Ввожу 5 символов. Затем я перезапускаю код и в...  other
 35068  Код вообще недорабатан. Вам стоит пройтись по ...  other
 70829  как не причина если эта покраска накладывает о...  other
 12467  Ну и есл